In [150]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import geone
import geone.covModel as gcm
import shutil

sys.path.append("../../../../")
import ArchPy

## Exo 1

In [151]:
# exo 1

#grid
sx = 0.5
sy = 1
sz = 0.15
x0 = 0
y0 = 0
z0 = -15
nx = 200
ny = 1
nz = 100

dimensions = (nx, ny, nz)
spacing = (sx, sy, sz)
origin = (x0, y0, z0)  


# units
#units covmodel
covmodelB = gcm.CovModel2D(elem=[('cubic', {'w':.5, 'r':[40, 40]})])
covmodelA = gcm.CovModel2D(elem=[('cubic', {'w':8, 'r':[30, 30]})])

## facies covmodel
covmodel_SIS_A = gcm.CovModel3D(elem = [("exponential", {"w":.25,"r":[20, 20, 2]})], alpha=0, name="vario_SIS") # input variogram
covmodel_SIS_B = gcm.CovModel3D(elem = [("exponential", {"w":.25,"r":[20, 20, 2]})], alpha=0, name="vario_SIS") # input variogram


#create Lithologies 
C = ArchPy.base.Unit(name = "C", order = 1, ID = 1, color = "lightblue", contact = "onlap",
                     surface = ArchPy.base.Surface(contact = "onlap"))


dic_s_B = {"int_method" : "grf_ineq","covmodel" : covmodelB, "mean" : -5}
dic_f_B = {"f_method" : "SIS", "neig" : 20, "f_covmodel":covmodel_SIS_B}
B = ArchPy.base.Unit(name = "B", order = 2, ID = 2, color="gold", contact = "onlap",
                     surface = ArchPy.base.Surface(contact = "erode", dic_surf = dic_s_B),
                     dic_facies = dic_f_B)


dic_s_A = {"int_method" : "grf_ineq","covmodel" : covmodelA, "mean":-9}
dic_f_A = {"f_method" : "SIS","neig" : 20, "f_covmodel" : covmodel_SIS_A}
A = ArchPy.base.Unit(name="A", order = 3,ID = 3, color="blue", contact = "onlap",
                     surface = ArchPy.base.Surface(dic_surf=dic_s_A, contact="erode"),
                     dic_facies = dic_f_A)

P1 = ArchPy.base.Pile("P1")
P1.add_unit([C, B, A])

Unit C: Surface added for interpolation
Unit B: covmodel for SIS added
Unit B: Surface added for interpolation
Unit A: covmodel for SIS added
Unit A: Surface added for interpolation
Stratigraphic unit C added
Stratigraphic unit B added
Stratigraphic unit A added


In [152]:
# covmodels for the property model
covmodelK = gcm.CovModel3D(elem=[("exponential",{"w":0.3,"r":[5,5,1]})],alpha=-20,name="K_vario")
covmodelK2 = gcm.CovModel3D(elem=[("spherical",{"w":0.1,"r":[3,3,1]})],alpha=0,name="K_vario_2")
covmodelPoro = gcm.CovModel3D(elem=[("exponential",{"w":0.005,"r":[10,10,10]})],alpha=0,name="poro_vario")

sand = ArchPy.base.Facies(ID = 1,name="Sand",color="yellow")
gravel = ArchPy.base.Facies(ID = 2,name="Gravel",color="lightgreen")
clay = ArchPy.base.Facies(ID = 4,name="Clay",color="blue")
silt = ArchPy.base.Facies(ID = 6,name="Silt",color="goldenrod")

C.add_facies([silt])
B.add_facies([sand, gravel])
A.add_facies([silt, clay])

permea = ArchPy.base.Prop("K",[sand, gravel, clay, silt],
                  [covmodelK2, covmodelK, covmodelK, covmodelK2],
                  means=[-3.5,-2,-8,-6.5],
                  int_method = ["sgs","sgs","sgs","sgs"],
                  def_mean=-5)

Facies Silt added to unit C
Facies Sand added to unit B
Facies Gravel added to unit B
Facies Silt added to unit A
Facies Clay added to unit A


In [153]:
T1 = ArchPy.base.Arch_table(name = "exo1",seed= 4)
T1.set_Pile_master(P1)
T1.add_grid(dimensions, spacing, origin)
T1.rem_all_bhs()
T1.add_prop([permea])

Pile sets as Pile master
## Adding Grid ##
## Grid added and is now simulation grid ##
boreholes removed
Property K added


In [154]:
import copy
T1_copy = copy.deepcopy(T1)

In [155]:
T1.process_bhs()

##### ORDERING UNITS ##### 
Pile P1: ordering units
Stratigraphic units have been sorted according to order
hierarchical relations set
No borehole found - no hd extracted


In [156]:
T1.compute_surf(1)
T1.compute_facies()
T1.compute_prop()

Boreholes not processed, fully unconditional simulations will be tempted
########## PILE P1 ##########
Pile P1: ordering units
Stratigraphic units have been sorted according to order

#### COMPUTING SURFACE OF UNIT A
A: time elapsed for computing surface 0.005003929138183594 s

#### COMPUTING SURFACE OF UNIT B
B: time elapsed for computing surface 0.00900888442993164 s

#### COMPUTING SURFACE OF UNIT C
C: time elapsed for computing surface 0.0 s

Time elapsed for getting domains 0.0010013580322265625 s
##########################


### 0.016015052795410156: Total time elapsed for computing surfaces ###

### Unit C: facies simulation with homogenous method ####
### Unit C - realization 0 ###
Time elapsed 0.0 s

### Unit B: facies simulation with SIS method ####
### Unit B - realization 0 ###
Only one facies covmodels for multiples facies, adapt sill to right proportions
Time elapsed 0.02 s

### Unit A: facies simulation with SIS method ####
### Unit A - realization 0 ###
Only one facies 

C:\Users\emmal\switchdrive\phenix\notebooks\examples\Exos Phenix\exo 1\../../../..\ArchPy\base.py:6695: RuntimeWarning: invalid value encountered in long_scalars
  p=np.sum(facies == fa.ID)/len(facies == fa.ID) #calculate proportion of facies


In [157]:
np.random.seed(54)

nbh = 5
posx = np.random.random(size=nbh) * T1.xg[-1]
posy = np.ones([nbh]) * 0.5
boreholes = T1.make_fake_bh(posx, posy)[0][0]

In [158]:
ID = 1

for i in boreholes:
    
    i.ID = ID
    ID += 1

In [159]:
T1_copy.add_bh(boreholes)

Borehole 1 goes below model limits, borehole 1 depth cut
Borehole 1 added
Borehole 2 goes below model limits, borehole 2 depth cut
Borehole 2 added
Borehole 3 goes below model limits, borehole 3 depth cut
Borehole 3 added
Borehole 4 goes below model limits, borehole 4 depth cut
Borehole 4 added
Borehole 5 goes below model limits, borehole 5 depth cut
Borehole 5 added


In [160]:
T1_copy.process_bhs()

##### ORDERING UNITS ##### 
Pile P1: ordering units
Stratigraphic units have been sorted according to order
hierarchical relations set

 ## Computing distributions for Normal Score Transform ##

Processing ended successfully


In [161]:
T1_copy.plot_bhs("facies")

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [162]:
T1.plot_facies()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [163]:
ArchPy.inputs.write_bh_files(T1_copy)

('exo1.lbh', 'exo1.ud', 'exo1.fd')

In [164]:

for file in os.listdir(T1_copy.ws):
    shutil.copyfile(os.path.join(T1_copy.ws, file), os.path.join("data", file))
    os.remove(os.path.join(T1_copy.ws, file))

In [165]:
os.listdir("ArchPy_workspace")

[]